# FuzzyCocoPython quickstart

This notebook gives a compact tour of the scikit-learn style API exposed by `fuzzycocopython`.
We train a classifier, a regressor, inspect generated fuzzy rules, and show how to persist a model.

We will:
- fit a `FuzzyCocoClassifier` on the Iris dataset and review its fuzzy knowledge base
- fit a `FuzzyCocoRegressor` on the Diabetes dataset and inspect predictions
- compute rule activation statistics
- save and reload a trained estimator

In [1]:
from pathlib import Path

import pandas as pd
from sklearn.datasets import load_diabetes, load_iris
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split

from fuzzycocopython import FuzzyCocoClassifier, FuzzyCocoRegressor

RANDOM_STATE = 0

## Classification example

In [2]:
iris = load_iris(as_frame=True)
X_iris = iris.data
y_iris = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X_iris,
    y_iris,
    test_size=0.3,
    stratify=y_iris,
    random_state=RANDOM_STATE,
)

In [3]:
clf = FuzzyCocoClassifier(nb_rules=10, input_vars_params__nb_sets=9, nb_max_var_per_rule=12, random_state=RANDOM_STATE)
clf.fit(X_train, y_train)

,params,None
,random_state,0
,params_overrides,"{'global_params': {'nb_max_var_per_rule': 12, 'nb_rules': 10}, 'input_vars_params': {'nb_sets': 9}}"


In [4]:
iris_predictions = pd.DataFrame(
    {
        "y_true": y_test.reset_index(drop=True),
        "y_pred": clf.predict(X_test),
    }
)

print(f"Accuracy: {clf.score(X_test, y_test):.3f}")

Accuracy: 0.667


### Inspect the fuzzy system

In [5]:
# Each row corresponds to a fuzzy rule described by the engine
clf.rules_df_.head()

,rule,role,io,var,label,position,orig_set
0,default,default,output,target,High,0.745098,target.2
1,rule1,antecedent,input,petal width (cm),Set 2,0.570588,petal width (cm).2
2,rule1,consequent,output,target,Low,0.439216,target.1
3,rule2,antecedent,input,petal width (cm),Set 3,0.730588,petal width (cm).3
4,rule2,consequent,output,target,Low,0.439216,target.1


In [6]:
rule_stats = clf.rules_stat_activations(X_test)
rule_stats.head()

,mean,std,min,max,usage_rate,usage_rate_pct,importance_pct,impact_pct
"IF (petal width (cm) is Set 1), THEN (target is Low)",0.333333,0.471405,0.0,1.000000,0.333333,33.333333,33.333333,11.111111
"IF (petal length (cm) is Set 5), THEN (target is High)",0.203958,0.335459,0.0,0.982189,0.288889,28.888889,27.839142,8.042419
"IF (petal width (cm) is Set 3), THEN (target is Low)",0.008095,0.037536,0.0,0.182143,0.044444,4.444444,1.049747,0.046655
"IF (petal width (cm) is Set 2), THEN (target is Low)",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
# simply look at the learned rules
clf.rules_, clf.default_rules_

([IF (petal width (cm) is Set 2), THEN (target is Low),
  IF (petal width (cm) is Set 3), THEN (target is Low),
  IF (petal width (cm) is Set 1), THEN (target is Low),
  IF (petal length (cm) is Set 5), THEN (target is High)],
 [ELSE (target is High)])

## Regression example

In [8]:
diabetes = load_diabetes(as_frame=True)
X_reg = diabetes.data
y_reg = diabetes.target

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg,
    y_reg,
    test_size=0.25,
    random_state=RANDOM_STATE,
)

In [9]:
reg = FuzzyCocoRegressor(random_state=RANDOM_STATE)
reg.fit(X_train_reg, y_train_reg)

,params,None
,random_state,0
,params_overrides,None


In [10]:
reg_predictions = reg.predict(X_test_reg)
print(f"R^2 score: {reg.score(X_test_reg, y_test_reg):.3f}")
print(f"RMSE: {root_mean_squared_error(y_test_reg, reg_predictions):.2f}")

pd.DataFrame(
    {
        "y_true": y_test_reg.reset_index(drop=True),
        "y_pred": reg_predictions,
    }
).head()

R^2 score: -0.427
RMSE: 84.18


,y_true,y_pred
0,321.0,106.823529
1,215.0,106.823529
2,127.0,109.764009
3,64.0,115.119830
4,175.0,119.723612


In [11]:
reg.rules_df_.head()

,rule,role,io,var,label,position,orig_set
0,default,default,output,target,Low,106.823529,target.1
1,rule1,antecedent,input,s5,Low,-0.116931,s5.1
2,rule1,consequent,output,target,High,135.776471,target.2
3,rule2,antecedent,input,s1,High,0.147730,s1.2
4,rule2,antecedent,input,s3,High,0.073343,s3.2


## Persisting a trained model

In [12]:
artifact = Path("fuzzycoco_classifier.pkl")
clf.save(artifact)
loaded_clf = FuzzyCocoClassifier.load(artifact)

print("Predictions from the reloaded model:")
print(loaded_clf.predict(X_test.iloc[:3]))

artifact.unlink(missing_ok=True)

Predictions from the reloaded model:
[1 1 0]
